<a href="https://colab.research.google.com/github/airpods69/DeepLearningGrind/blob/main/MangaFacialExpressions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Useless stuff Needed to be done for kaggle

In [5]:
!cp kaggle.json /root/.kaggle/

In [6]:
!kaggle datasets download -d mertkkl/manga-facial-expressions

 53% 9.00M/17.0M [00:00<00:00, 93.8MB/s]
100% 17.0M/17.0M [00:00<00:00, 146MB/s] 


In [7]:
!ls

kaggle.json  manga-facial-expressions.zip  sample_data


In [ ]:
!unzip manga-facial-expressions.zip -d manga-facial-expressions/

### Real Work time

In [10]:
!ls

kaggle.json		  manga-facial-expressions.zip
manga-facial-expressions  sample_data


In [11]:
train_path = "./manga-facial-expressions"
test_path = "./manga-facial-expressions"

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [13]:
datagen = ImageDataGenerator(rescale=1/255.)

In [14]:
train = datagen.flow_from_directory(train_path,
                                    target_size=(48,48),
                                    class_mode="sparse",
                                    seed=1,
                                    color_mode="grayscale",
                                    batch_size=128)

test = datagen.flow_from_directory(test_path,
                                    target_size=(48,48),
                                    class_mode="sparse",
                                    seed=1,
                                    color_mode="grayscale",
                                    batch_size=128)

Found 455 images belonging to 7 classes.
Found 455 images belonging to 7 classes.


In [15]:
print(train.class_indices)
emotion = {0: "angry", 
           1: "crying", 
           2: "embarassed", 
           3: "happy", 
           4: "pleased", 
           5: "sad", 
           6: "shock"}

{'angry': 0, 'crying': 1, 'embarrassed': 2, 'happy': 3, 'pleased': 4, 'sad': 5, 'shock': 6}


In [16]:
x, y = train.next()
print(x.shape, y.shape)
train.reset()

(128, 48, 48, 1) (128,)


In [17]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, LeakyReLU, Dense, Dropout
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [28]:
model = Sequential()

# Layer 1
model.add(Conv2D(filters = 32,
                 kernel_size = (3,3),
                 padding = 'same',
                 input_shape = (48, 48, 1)
                 ))

model.add(LeakyReLU())

model.add(Conv2D(filters = 32,
                 kernel_size = (3,3),
                 padding = 'same'
                 ))

model.add(LeakyReLU())

model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Dropout(0.2))

# Layer 2
model.add(Conv2D(filters = 64,
                 kernel_size = (3,3),
                 padding = 'same',
                 ))

model.add(LeakyReLU())

model.add(Conv2D(filters = 64,
                 kernel_size = (3,3),
                 padding = 'same'
                 ))

model.add(LeakyReLU())

model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Dropout(0.3))

# Layer 3
model.add(Conv2D(filters = 128,
                 kernel_size = (3,3),
                 padding = 'same'
                 ))

model.add(LeakyReLU())

model.add(Conv2D(filters = 128,
                 kernel_size = (3,3),
                 padding = 'same'
                 ))

model.add(LeakyReLU())

model.add(Conv2D(filters = 128,
                 kernel_size = (3,3),
                 padding = 'same'
                 ))

model.add(LeakyReLU())

model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Dropout(0.4))

model.add(Flatten())

model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.4))

model.add(Dense(len(train.class_indices), activation = 'softmax'))

model.summary()
model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=1e-3), 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 48, 48, 32)        320       
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 48, 48, 32)        0         
                                                                 
 conv2d_11 (Conv2D)          (None, 48, 48, 32)        9248      
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 48, 48, 32)        0         
                                                                 
 batch_normalization_4 (Batc  (None, 48, 48, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 24, 24, 32)       0         
 2D)                                                  

In [29]:
early_stopping = EarlyStopping(monitor = 'val_accuracy',
                               min_delta = 0.00005,
                               patience=11,
                               verbose=1,
                               restore_best_weights=True)

lr_scheduler = ReduceLROnPlateau(monitor = 'val_accuracy',
                                 factor = 0.5,
                                 patience = 7,
                                 min_lr = 1e-7,
                                 verbose = 1)

callbacks = [
             early_stopping,
             lr_scheduler
]

In [30]:
model.fit(train, 
          validation_data = test,
          epochs = 40,
          callbacks = callbacks)

Epoch 1/40
4/4 [==============================] - 4s 608ms/step - loss: 10.1177 - accuracy: 0.1429 - val_loss: 2.0907 - val_accuracy: 0.1143 - lr: 0.0010
Epoch 2/40
4/4 [==============================] - 1s 399ms/step - loss: 4.6772 - accuracy: 0.1824 - val_loss: 1.9072 - val_accuracy: 0.2242 - lr: 0.0010
Epoch 3/40
4/4 [==============================] - 1s 404ms/step - loss: 2.4091 - accuracy: 0.2132 - val_loss: 1.9222 - val_accuracy: 0.1846 - lr: 0.0010
Epoch 4/40
4/4 [==============================] - 2s 522ms/step - loss: 1.9694 - accuracy: 0.2264 - val_loss: 1.9211 - val_accuracy: 0.2286 - lr: 0.0010
Epoch 5/40
4/4 [==============================] - 2s 441ms/step - loss: 1.9855 - accuracy: 0.2022 - val_loss: 1.9259 - val_accuracy: 0.1890 - lr: 0.0010
Epoch 6/40
4/4 [==============================] - 1s 400ms/step - loss: 1.8523 - accuracy: 0.2571 - val_loss: 1.9150 - val_accuracy: 0.2176 - lr: 0.0010
Epoch 7/40
4/4 [==============================] - 1s 400ms/step - loss: 1.9071 - 

In [31]:
model.save('63.h5')

In [32]:
import numpy as np
x,y = test.next()
preds = model.predict(x)
idx = np.argmax(preds, axis = 1)
# idx.shape
print('correct prediction:', np.sum((y == idx)*1)/128)

correct prediction: 0.1640625


In [35]:
import matplotlib.pyplot as plt

In [37]:
# fig = plt.figure(1, (14, 14))

# k = 0
# for j in range(49):
#     px = x[j]
#     k += 1
#     ax = plt.subplot(7, 7, k)
#     ax.imshow(px, cmap='gray')
#     ax.set_xticks([])
#     ax.set_yticks([])
    
#     if  emotion_dict[y[j]] == emotion_dict[idx[j]]:
#         for axis in ['top','bottom','left','right']:
#             ax.spines[axis].set_linewidth(5)
#             ax.spines[axis].set_color('green')
#         ax.set_title(emotion_dict[idx[j]])

#     else:
#         for axis in ['top','bottom','left','right']:
#             ax.spines[axis].set_linewidth(5)
#             ax.spines[axis].set_color('red')
#         ax.set_title('P:'+emotion_dict[idx[j]]+" C:"+emotion_dict[y[j]])
#     plt.tight_layout()